# Old-school feature importance 👵👴

In [1]:
#@title Python library imports go here
import warnings
import multiprocessing as mp
from functools import partial
from typing import Tuple

import numpy as np
import pandas as pd
import numpy.typing as npt
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score


In [2]:
SEED = 42
np.random.seed(SEED)
warnings.filterwarnings('ignore')

## Part 1: Defining the model and dataset

**Question 1:** Load *diabetes* dataset from [sklearn](https://scikit-learn.org/stable/user_guide.html) and split the dataset into training (70%) and test set (30%)


In [3]:
dataset = load_diabetes()
X = dataset.data
y = dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)
print(f"Train data shape {X_train.shape}")
print(f"Test data shape {X_test.shape}")

Train data shape (309, 10)
Test data shape (133, 10)


**Question 2:** Define and train a random forest (RF) using sklearn on the training set.

In [4]:
estimators = 100
forest = RandomForestRegressor(n_estimators=estimators, random_state=SEED, oob_score=True)
forest.fit(X_train, y_train)

RandomForestRegressor(oob_score=True, random_state=42)

**Question 3:** Compute the RMSE error of your RF (on test set) and compare it to OOB RMSE error (on train set).

In [5]:
error_rmse = root_mean_squared_error(y_test, forest.predict(X_test))
print(f'RMSE: {error_rmse:.2f}')
oob_rmse = root_mean_squared_error(forest.oob_prediction_, y_train)
print(f'OOB RMSE error: {oob_rmse:.2f}')

RMSE: 53.48
OOB RMSE error: 59.23


**Question 4:** Plot the RMSE (on test) and OOB RMSE (on train) with respect to the number of trees (n_estimators) in RF.

As we train a pretty plenty of forests we just can make it parallel.

In [6]:
def train_forest(n_trees: int, n_features: int = 1) -> Tuple:
    forest = RandomForestRegressor(n_trees, random_state=SEED, oob_score=True, max_features=n_features)
    forest.fit(X_train, y_train)
    error_rmse = root_mean_squared_error(y_test, forest.predict(X_test))
    oob_error_rmse = root_mean_squared_error(forest.oob_prediction_, y_train)
    return error_rmse, oob_error_rmse

In [7]:
def train_by_increasing_forests_sizes(trees_in_forests: npt.NDArray[np.int_], pool, max_features: int = 1) -> Tuple:
    """
    Takes a pool of processes and use it to train in paraller forests with different sizes.
    """
    error_pairs = [i for i in tqdm(pool.imap_unordered(partial(train_forest, n_features=max_features), np.arange(1, 101)), total=trees_in_forests.shape[0])]
    errors = np.ascontiguousarray(error_pairs)
    return errors[:, 0], errors[:, 1]

In [8]:
trees_in_forests = np.arange(1, 101)
with mp.Pool(mp.cpu_count()) as pool:
    normal_error, oob_error = train_by_increasing_forests_sizes(trees_in_forests, pool)

  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
trace_1 = go.Scatter(
    x=trees_in_forests,
    y=normal_error,
    mode='lines',
    name="RMSE test error"
)

trace_2 = go.Scatter(
    x=trees_in_forests,
    y=oob_error,
    mode='lines',
    name="OOB RMSE error",
    yaxis='y2'
)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace_1)
fig.add_trace(trace_2,secondary_y=True)
fig.show()

**Question 5:** Plot the OOB RMSE error (on train) where max_features $\in \{1, \ldots, 10\}$ with respect to the number of trees in RF. Produce a single plot with multiple curves, each corresponding to a different max_features value.

In [10]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
with mp.Pool(mp.cpu_count()) as pool:
    for i, features_count in enumerate(tqdm(np.arange(1, 11))):
        _, oob_error = train_by_increasing_forests_sizes(trees_in_forests, pool, features_count)
        if i == 0:
            trace = go.Scatter(
                x=trees_in_forests,
                y=oob_error,
                mode='lines',
                name=f"OOB RMSE features {features_count}"
            )
            fig.add_trace(trace)
        else:
            trace = go.Scatter(
                x=trees_in_forests,
                y=oob_error,
                mode='lines',
                name=f"OOB RMSE features {features_count}",
                yaxis='y2'
            )
            fig.add_trace(trace, secondary_y=True)
fig.show()


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

## Part 2.1: Feature importance using MDI (Mean Decrease in Impurity)

**Question 1:** How to extract the decrease in impurity of each features from a given decision tree using sklearn. **Print it!**

In [11]:
tree = DecisionTreeRegressor(random_state=SEED)
tree.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [12]:
feature_names = dataset.feature_names

In [13]:
feature_importances = tree.feature_importances_

# Print the impurity of each feature
for name, importance in zip(feature_names, feature_importances):
    print(f"Feature: {name}, Importance (Impurity Reduction): {importance:.4f}")

Feature: age, Importance (Impurity Reduction): 0.0554
Feature: sex, Importance (Impurity Reduction): 0.0033
Feature: bmi, Importance (Impurity Reduction): 0.4431
Feature: bp, Importance (Impurity Reduction): 0.1004
Feature: s1, Importance (Impurity Reduction): 0.0674
Feature: s2, Importance (Impurity Reduction): 0.0676
Feature: s3, Importance (Impurity Reduction): 0.0340
Feature: s4, Importance (Impurity Reduction): 0.0340
Feature: s5, Importance (Impurity Reduction): 0.1332
Feature: s6, Importance (Impurity Reduction): 0.0616


**Question 2:** extract from the random forest the mean decrease in impurity (MDI) for all features. **Print it!**

In [14]:
# Print the impurity of each feature
for name, importance in zip(feature_names, forest.feature_importances_):
    print(f"Feature: {name}, Importance (Impurity Reduction): {importance:.4f}")

Feature: age, Importance (Impurity Reduction): 0.0586
Feature: sex, Importance (Impurity Reduction): 0.0111
Feature: bmi, Importance (Impurity Reduction): 0.4000
Feature: bp, Importance (Impurity Reduction): 0.1048
Feature: s1, Importance (Impurity Reduction): 0.0492
Feature: s2, Importance (Impurity Reduction): 0.0471
Feature: s3, Importance (Impurity Reduction): 0.0617
Feature: s4, Importance (Impurity Reduction): 0.0294
Feature: s5, Importance (Impurity Reduction): 0.1666
Feature: s6, Importance (Impurity Reduction): 0.0714


**Question 4:** plot the feature importances using the MDI

In [16]:
fig = px.bar(
    x=feature_names,
    y=feature_importances,
    labels={'x': 'Feature', 'y': 'Importance'},
    title='Feature Importance of Decision Tree'
)
fig.show()

## Part 2.2: Feature importance using MDA (Mean Decrease in Accuracy)

**Question 1:** implement the pseudocode of the MDA method using as accuracy measurement the OOB MSE.

**Question 2:** plot the feature importances using your implementation of MDA

**Question 2:** plot feature importances using permutation_importance() function of sklearn

**Question 3:** compare your implementation outputs to the permutation_importance() function of sklearn. Why is it different?

## BONUS: Correlated input variables



**Question 1:** sample the following Gaussian vector $(X_1, X_2, Y) \sim N(0, \mathbf{\Sigma})$ where $\mathbf{\Sigma} := \begin{pmatrix}
C & \mathbf{\tau}  \\
\mathbf{\tau}^\top & 1 \\  
\end{pmatrix}$,  $C := \begin{pmatrix}
1 & c  \\
c & 1 \\  
\end{pmatrix}$ and $\mathbf{\tau} := (\tau_0, \tau_0) \in \mathbb{R}^2$ where $\tau_0 \in (-1,1), \, c \in \mathbb{R}$.

**Question 2:** define a RF predictor on the previous data and compute the OOB MSE

**Question 3:** compute the permutation importance for $X_1$ and $X_2$ using the previous code with MSE error

**Question 3:** plot the theoretical MDA $I(X_j) := 2 \left( \frac{\tau_0}{1 + c} \right)^2$, with a fixed $\tau_0 = 0.5$, by varying the feature correlation $c$

**Question 4:** plot the empirical MDA *v.s.* theoretical MDA with respect to the feature correlation $c$